# Worm Communication

In [1]:
import serial
import ipywidgets
import threading
from ipywidgets import widgets
import time 
from IPython.display import display

#### Serial communication 

In [2]:
baudrate = 9600 # default, might need to adjust
serial_port = '/dev/tty.HC-06-DevB-1' # connect to the new hc-06 board
ser = serial.Serial(serial_port, baudrate, timeout=1) # open connection

#### Interface

In [2]:
button = widgets.Button(description = "Ho")

def clicked(a):
    ser.write("X".encode())
    

console_style = "style=\"width: 500px; color: steelblue; border-left: 6px solid darkslateblue; background-color: lightblue;\""
output = widgets.HTML(
    value="<dir "+console_style+"> empty </dir>",
    placeholder='Empty',
    description='Console:',
    disabled=True
)

command = widgets.Text(
    value='',
    placeholder='Enter command',
    description='Command:',
    disabled=False
)

def on_command_change(change):
    if (command.value != ''):
        ser.write(command.value.encode())
    command.value = ''

orientation = widgets.SelectionSlider(
    options=['forward', 'idle', 'backward'],
    value='idle',
    description='Orientation',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

direction = widgets.SelectionSlider(
    options=['left', 'neutral', 'right'],
    value='neutral',
    description='direction',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

def on_orientation_change(change):
    #if (command.value != ''):
    if (orientation.value == "forward"):
        ser.write("f".encode())
    elif (orientation.value == "backward"):
        ser.write("b".encode())
    else:
        ser.write("n".encode())
    
def on_direction_change(change):
    #if (command.value != ''):
    if (direction.value == "left"):
        ser.write("l".encode())
    elif (direction.value == "right"):
        ser.write("r".encode())
    else:
        ser.write("m".encode())

#### Receiving from worm

In [60]:
def handle_data(data, text_list):
    # print(data)
    text_list = text_list[1:]+[time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())+"|||"+data]
    display_text = ""
    for x in text_list:
        display_text += x + "<br>"
    output.value = "<dir "+console_style+">" + display_text + "</dir>"
    return text_list

def read_from_port(ser):
    text_list = ["-\n","-\n","-\n","-\n","-\n"]
    while True:
        reading = ser.readline().decode()
        text_list = handle_data(reading, text_list)

### Start receiving thread; Display interface

In [5]:
thread = threading.Thread(target=read_from_port, args=(ser,))
thread.start()

In [3]:
display(button)
display(orientation)
display(direction)
display(command)
display(output)
#button.on_click(clicked)
command.observe(on_command_change, names='value')
orientation.observe(on_orientation_change, names='value')
direction.observe(on_direction_change, names='value')